In [1]:
import sys
sys.path.append("..")

In [2]:
import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import ast

torch.manual_seed(1)
import pandas as pd
import numpy as np
import gensim
import torchtext
from torchtext import data
from torchtext import vocab
from utils.db_client import DBClient

In [4]:
# Make DB Connection
db = DBClient(secrets_path='../configs/db_secrets.json')

I'm trying
Connected to political tweets DB


In [96]:
# Load data
examples = db.read("select tweet_text_clean, label from staging.train_twitter140 order by Random() limit 20 ")

In [97]:
# Define Fields
txt_field = data.Field(sequential=True, 
                       include_lengths=True, 
                       use_vocab=True)
label_field = data.Field(sequential=False, 
                         use_vocab=False, 
                         pad_token=None, 
                         unk_token=None)
train_val_fields = [
    ('SentimentText', txt_field), # process it as text
    ('Sentiment', label_field) # process it as label
]

# Convert text ecamples to Example datatype
examples = [data.Example.fromlist((ast.literal_eval(example[0]), [example[1]]), train_val_fields) for example in examples]

# Create dataset
dataset = data.Dataset(examples, train_val_fields)

# Make vocab for text and label fields
txt_field.build_vocab(dataset)
label_field.build_vocab(dataset)

# Make train/val/test splits
train_data, valid_data, test_data = dataset.split([0.8, 0.1, 0.1])

In [98]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 16
Number of validation examples: 2
Number of testing examples: 2


In [108]:
# Make data iterators
BATCH_SIZE = 2

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, val_iterator, test_iterator = data.BucketIterator.splits((train_data, valid_data, test_data), 
                                                          batch_sizes = (BATCH_SIZE, BATCH_SIZE, BATCH_SIZE))

In [82]:
# Lod pretrained vectors and create embeddings
vectors = vocab.Vectors('glove.twitter.27B.50d.txt', '../model/glove.twitter.27B/') 
txt_field.vocab.set_vectors(vectors.stoi, vectors.vectors, vectors.dim)

embedding = nn.Embedding.from_pretrained(torch.FloatTensor(txt_field.vocab.vectors))

100%|█████████▉| 1192069/1193514 [01:00<00:00, 26239.09it/s]

In [83]:
# def prepare_sequence(seq, to_ix):
#     idxs = [to_ix[w] for w in seq]
#     return torch.tensor(idxs, dtype=torch.long)

# word_to_ix = {}
# for tweet, tag in examples:
#     for word in tweet:
#         if word not in word_to_ix:
#             word_to_ix[word] = len(word_to_ix)
# print(word_to_ix)
# tag_to_ix = {0: 0,  4: 1}

In [84]:
class LSTMSentiment(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size, embedding):
        super(LSTMSentiment, self).__init__()
        self.hidden_dim = hidden_dim
        
        #self.weights = torch.FloatTensor(model.wv.vectors)
        self.word_embeddings = embedding

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def training(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.SentimentText[0]).squeeze(1)
        
        loss = criterion(predictions, batch.Sentiment)
        
        acc = binary_accuracy(predictions, batch.Sentiment)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.SentimentText[0]).squeeze(1)
            
            loss = criterion(predictions, batch.Sentiment)
            
            acc = binary_accuracy(predictions, batch.Sentiment)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [128]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [129]:
lstm_model = LSTMSentiment(100, 2, len(txt_field.vocab), len(label_field.vocab), embedding)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(lstm_model.parameters(), lr=0.1)
criterion = nn.BCEWithLogitsLoss()

N_EPOCHS = 2

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = training(lstm_model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(lstm_model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

ValueError: Target size (torch.Size([2, 1])) must be the same as input size (torch.Size([6, 2]))

In [130]:
for batch in train_iterator:
    print(batch.Sentiment)
    print()

tensor([[0],
        [4]])

tensor([[4],
        [0]])

tensor([[4],
        [0]])

tensor([[0],
        [0]])

tensor([[4],
        [0]])

tensor([[4],
        [0]])

tensor([[4],
        [0]])

tensor([[4],
        [0]])



In [36]:


training_data = dataset

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = lstm_model(inputs)
    print(tag_scores)

for epoch in range(2):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        lstm_model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        #sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = lstm_model(sentence)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = lstm_model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([[-0.7577, -0.6325],
        [-0.7577, -0.6325],
        [-0.7702, -0.6216],
        [-0.7546, -0.6352],
        [-0.7543, -0.6355],
        [-0.7630, -0.6278],
        [-0.7633, -0.6276],
        [-0.7692, -0.6225]])


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys


TypeError: embedding(): argument 'indices' (position 2) must be Tensor, not list

In [ ]:
lstm_model

In [48]:
model